# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/minaferizbegovic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/minaferizbegovic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/minaferizbegovic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster', engine) 
df.head()
X = df['message']
Y = df.iloc[:,4:40]

#res = np.sum(np.array(Y), axis = 1)
#min(res)

### 2. Write a tokenization function to process your text data

In [32]:
def tokenize(text):
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('mrf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)


/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x304e03160>)),
                ('tfidf', TfidfTransformer()),
                ('mrf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
y_pred = pipeline.predict(X_test)
for ind, column in enumerate(y_test.columns):
    print(classification_report(np.array(y_test[column]), y_pred[:,ind]))

              precision    recall  f1-score   support

           0       0.70      0.33      0.45      1523
           1       0.82      0.96      0.89      4984

    accuracy                           0.81      6507
   macro avg       0.76      0.64      0.67      6507
weighted avg       0.80      0.81      0.78      6507

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5406
           1       0.85      0.48      0.62      1101

    accuracy                           0.90      6507
   macro avg       0.88      0.73      0.78      6507
weighted avg       0.89      0.90      0.89      6507

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6474
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

              preci

/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [36]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x304e03160>)),
  ('tfidf', TfidfTransformer()),
  ('mrf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x304e03160>),
 'tfidf': TfidfTransformer(),
 'mrf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_i

### 6. Improve your model
Use grid search to find better parameters. 

In [37]:
from sklearn.model_selection import GridSearchCV
parameters = {'mrf__estimator__n_estimators': [10, 20]}

#['estimator__bootstrap', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs']

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/udacity/lib/python3.8/site-packages/sklearn/featu

In [39]:
(y_pred==y_test).mean()

related                   0.809897
request                   0.894114
offer                     0.994929
aid_related               0.756109
medical_help              0.920701
medical_products          0.946365
search_and_rescue         0.971262
security                  0.981097
military                  0.969264
child_alone               1.000000
water                     0.952359
food                      0.932995
shelter                   0.928385
clothing                  0.987706
money                     0.977409
missing_people            0.987398
refugees                  0.963731
death                     0.960043
other_aid                 0.868757
infrastructure_related    0.937145
transport                 0.956047
buildings                 0.951130
electricity               0.978024
tools                     0.994160
hospitals                 0.989550
shops                     0.996926
aid_centers               0.990164
other_infrastructure      0.956355
weather_related     

In [40]:
cv.best_params_

{'mrf__estimator__n_estimators': 20}

### 9. Export your model as a pickle file

In [41]:
import pickle
pickle.dump(cv, open("cv.p", "wb")) 